# Momentum Strategies based on Jegadeesh and Titman 1993 "Returns to buying Winners and selling Losers: Implications for Stock Market Efficiency"
--------------------------------------------------------------------------------------------------------

**<i>Program designed and executed by  Aryan Ayyar and  Dr. Abhishek Rohit under the guidance of Dr. Madhu Veeraraghavan and Dr Kavitha Ranganathan**
***

## Data Import and Filtering -> Select the database and apply required filters

<div style="text-align: justify">Our program is based on WRDS Python Momentum code and can be found at <a href="https://wrds-www.wharton.upenn.edu/documents/1442/wrds_momentum_demo.html" target="_blank">this link</a>. The program is decided to mimick the overlapping portfolio construction used by Narasimhan Jegadeesh and Sheridan Titman as well as incorporates liquidity factors from Lee and Swaminathan (2001).As quoted in the 1993 paper "To increase the power of our tests, the strategies we examine include
portfolios with overlapping holding periods. Therefore, in any given month t,
the strategies hold a series of portfolios that are selected in the current
month as well as in the previous K - 1 months, where K is the holding
period. Specifically, a strategy that selects stocks on the basis of returns over
the past J months and holds them for K months (we will refer to this as a
J-month/K-month
strategy) is constructed as follows: At the beginning of
each month t the securities are ranked in ascending order on the basis of
their returns in the past J months. Based on these rankings, ten decile
portfolios (or quintiles or baskets as deemed fit according to the number of stocks) are formed that equally weight the stocks contained in the top
decile, the second decile, and so on. The top decile portfolio is called the
"losers" decile and the bottom decile is called the "winners" decile. In each
month t, the strategy buys the winner portfolio and sells the loser portfolio,
holding this position for K months. In addition, the strategy closes out the
position initiated in month t - K. Hence, under this trading strategy we
1
revise the weights on - of the securities in the entire portfolio in any given
month and carry over the rest from the previous month.".<i> * Please note that in Jegadeesh and Titman, the W-L portfolio is P10-P1 and not P1-P10, the wordings as quoted in the paper may be misleading </div>

In [148]:
#import the necessary files and modules
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
from scipy import stats
from pandas.tseries.offsets import *
import datetime as dt
import matplotlib.pyplot as plt
ex="andy"
#Use stocks only is Bombay Stock Exchange not present in National Stock Exchange
bse_only= False

#Liquidity decile as factored by Lee and Swaminathan-> "liquid","illiquid","None"
liquidity= None
#Make appropriate deciles and basked (3-10)
basket=10
decile=10

#Split into deciles based on Market Cap
market_decile=False
subsample= False
sampend=False

bottom =True
bottom_only=False
q=0.05

winsorize_turn=False
winsorize_ret=False
level=0.01

penny_rem=False
penny=1

rem_fin=True
rem_govt=True

In [149]:
#Use Andy's Compustat data
bse="BSE_.csv"
nse="NSE_.csv"
andy="andy_bse_merged.csv"

if ex=="bse":
    crsp= pd.read_csv(bse,header=0,parse_dates=True)
if ex=="nse":
    crsp= pd.read_csv(nse,header=0,parse_dates=True)
if ex=="andy":
    crsp=pd.read_csv(andy, usecols=['permno', 'Date', 'ret', 'Market Capitalization', 'turn','company_name','co_nic_code','nse_dummy','owner_gp_name'])
    crsp['nic_code_ini']=crsp['co_nic_code'].astype(str).str[:2]
    exclude= pd.read_excel('andy_bse_lookup.xlsx',sheet_name="Sheet2")
    
#Deals with Jayant Verma's monthly risk-free factors
jayant= pd.read_csv('Jayant Verma Monthly.csv',usecols=['date','RF_'],parse_dates=True)
jayant['date']=pd.to_datetime(jayant['date'],format="%Y-%m") + MonthEnd(0)
jayant=jayant.set_index('date')

C:\Users\aryan-intern23\AppData\Local\Temp\5\ipykernel_7136\3277890696.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp=pd.read_csv(andy, usecols=['permno', 'Date', 'ret', 'Market Capitalization', 'turn','company_name','co_nic_code','nse_dummy','owner_gp_name'])


In [150]:
if bse_only:
    crsp=crsp[crsp['nse_dummy']==0]
    
#Remove the data which has data < 2000
crsp['Date']= pd.to_datetime(crsp['Date']) + MonthEnd(0)
crsp=crsp[crsp['Date'].dt.year >=2000]    

crsp= crsp.rename(columns={'Date':'date'})

cond= crsp.groupby('permno')['turn'].apply(lambda x: x.isna().any()).reset_index().rename(columns={'turn':'exc'})
crsp= pd.merge(crsp,cond,on="permno")
#Exclude the firms which have NA in their turnover data
crsp=crsp[crsp['exc']==False]

#Convert the returns into a suitable format
crsp['ret']= crsp['ret']/100

if winsorize_ret:
    crsp['ret']=crsp.groupby('date')['ret'].transform(lambda x: winsorize(x,limits=[0.01,0.01],inplace=True))

# Final Calcultion through returns foramtion
#fill in missing return with 0
crsp['ret'] = crsp['ret'].fillna(0)

# create log return for future usage
crsp['logret'] = np.log(1+crsp['ret'])
#crsp=crsp.dropna(subset='logret')

crsp.columns=crsp.columns.str.strip()
###########################################
##Break here to describe the crsp dataframe
###########################################
#Remove the Government and financial firms for the analysis
if rem_govt:
    crsp= crsp[~crsp['owner_gp_name'].str.contains('Central')]
    crsp= crsp[~crsp['owner_gp_name'].str.contains('State')]

crsp['firm_threshold']= crsp.groupby(['date'])['permno'].transform(lambda x:x.nunique())
crsp=crsp[crsp['firm_threshold']> 30]

#Filter the Financial Firms with NIC_ini = 64,65,66
if rem_fin:
    crsp= crsp[~ crsp['nic_code_ini'].isin([64,65,66])]

if penny_rem:
    crsp=crsp[crsp['penny']==1]

if market_decile:
    crsp['market_decile']= crsp.groupby('date')['Market Capitalization'].transform(lambda x: pd.qcut(x, 10, labels=False)+1)
    crsp= crsp[~crsp['market_decile'].isin([1])]

if bottom:
    crsp['threshold']=crsp.groupby('date')['Market Capitalization'].transform(lambda x: x.quantile(q))
    crsp=crsp[crsp['Market Capitalization']> crsp['threshold']]
if bottom_only:
    crsp['threshold']=crsp.groupby('date')['Market Capitalization'].transform(lambda x: x.quantile(q))
    crsp=crsp[crsp['Market Capitalization']< crsp['threshold']]
    
#Create baskets based on the turnover ratio
crsp['basket']= crsp.groupby('date')['turn'].transform(lambda x: pd.qcut(x, basket, labels=False)+1)

#Pre filtering Block  
if liquidity=="liquid":
    crsp=crsp[crsp['basket']==basket]
    
if liquidity=="illiquid":
    crsp=crsp[crsp['basket']== 1]
    
if subsample:
    crsp= crsp[crsp['date']>= sub_date ]

crsp['ret_threshold']= crsp.groupby(['date'])['permno'].transform(lambda x:x.nunique())
#Filter the firms which have a minimum of 10 returns in a given months
crsp=crsp[crsp['ret_threshold']> 10]

## Momentum Analysis -> Post Initial Filtering, run the Momentum Strategy
--------------------------------------------------------------------------

In [151]:
def momentum_rets(crsp_m,jayant,j=6,k=1,gap=False,rem_excess=False,sub_risk_free=False):
    """
    Runs a momentum strategy in accordance with Jegadeesh and Titman (1993) paper titled
    "Returns to buying winners and selling losers" for the given parameters.
    """
    tmp_crsp = crsp_m[['permno','date','ret','logret']].sort_values(['permno','date']).set_index('date')

    # Calculate Rolling Cumulative Return in the Formation Period
    # By summing log returns over the J month formation period

    umd = tmp_crsp.groupby(['permno'])['logret'].rolling(j, min_periods=j).sum().reset_index()
    umd = umd.rename(columns={'logret':'sumlogret'})

    # Then exp the sum log return to get compound return (not necessary) 
    umd['cumret']=np.exp(umd['sumlogret'])-1
    #umd=umd[umd['cumret']!=0]

    # For each date: assign ranking 1-10 based on cumret
    # 1=lowest 10=highest cumret
    umd=umd.dropna(axis=0, subset=['cumret'])


    #Important Step to be followed

    if rem_excess:

        umd['thres']= umd.groupby('date')['cumret'].transform(lambda x:np.percentile(x,97.5))
        umd=umd.set_index(['permno','date'])
        aryan=umd[umd['cumret']<umd['thres']].groupby(level=1)['cumret'].transform(lambda x: pd.qcut(x,decile, labels=False)+1).to_frame(name="momr").reset_index()

        umd= pd.merge(umd,aryan,on=['date','permno'],how='outer')
        umd['momr']= umd['momr'].astype(str).str[:-2]
        umd=umd.reset_index()

    else:
        umd['momr']=umd.groupby('date')['cumret'].transform(lambda x: pd.qcut(x, decile, labels=False,duplicates='drop'))
        # shift momr from 0-9 to 1-10
        umd.momr=1+umd.momr

    # First lineup date to month end date medate
    # Then calculate hdate1 and hdate2 using medate
    # Holding Period Length: K can be between 3 to 12 months
    if gap:
        g=1
    else:
        g=0
    umd['form_date'] = umd['date']
    umd['medate'] = umd['date']+MonthEnd(0)
    umd['hdate1']=umd['medate']+MonthBegin(1+g)
    umd['hdate2']=umd['medate']+MonthEnd(k+g)
    umd = umd[['permno', 'form_date','momr','hdate1','hdate2']]

    port = pd.merge(crsp_m[['permno','date','ret']], umd, on=['permno'], how='inner')
    port = port[(port['hdate1']<=port['date']) & (port['date']<=port['hdate2'])]

    # Rearrange the columns;
    port = port[['permno','form_date', 'momr', 'hdate1','hdate2', 'date', 'ret']]

    umd_port = port.groupby(['date','momr', 'form_date'])['ret'].mean().reset_index()

    # Skip first two years of the sample 
    start_yr = umd_port.date.dt.year.min()+2
    umd_port = umd_port.loc[umd_port.date.dt.year>=start_yr]
    umd_port = umd_port.sort_values(by=['date','momr'])

    # Create one return series per MOM group every month
    ewret = umd_port.groupby(['momr','date'])['ret'].mean().reset_index()
    ewstd = umd_port.groupby(['momr','date'])['ret'].std().reset_index()

    ewret = ewret.rename(columns={'ret':'ewret'})
    ewstd = ewstd.rename(columns={'ret':'ewretstd'})

    ewretdf = pd.merge(ewret, ewstd, on=['date','momr'], how='inner')
    ewretdf = ewretdf.sort_values(by=['momr', 'date'])


    # Transpose portfolio layout to have columns as portfolio returns
    ewret_t = ewretdf.pivot(index='date', columns='momr', values='ewret')

    # Add prefix port in front of each column
    ewret_t = ewret_t.add_prefix('port')
    ewret_t = ewret_t.rename(columns={'port1':'losers', 'port'+ str(decile):'winners'})
    ewret_t['long_short'] = ewret_t.winners - ewret_t.losers

    #Extremely important step for decile formation
    ewret_t=ewret_t.dropna(axis=0,subset=['losers','winners'])
    # Compute Long-Short Portfolio Cumulative Returns

    ewret_t= ewret_t.join(jayant,on='date',how='inner')
    if sub_risk_free:
        ewret_t['long_short']= ewret_t.long_short - ewret_t.RF_

    ewret_t['cumret_winners']   = (1+ewret_t.winners).cumprod()-1
    ewret_t['cumret_losers']    = (1+ewret_t.losers).cumprod()-1
    ewret_t['cumret_long_short']= (1+ewret_t.long_short).cumprod()-1


    #################################
    # Portfolio Summary Statistics  #
    ################################# 

    # Mean 
    mom_mean = ewret_t[['winners', 'losers', 'long_short']].mean().to_frame().reset_index(drop=False)
    mom_mean = mom_mean.rename(columns={'index': 'momr',0:'mean'})


    mom_ser= pd.DataFrame(columns=['winners','losers','long_short'])
    mom_ser['winners']=pd.Series(stats.sem(ewret_t['winners']))
    mom_ser['losers']=pd.Series(stats.sem(ewret_t['losers']))
    mom_ser['long_short']= pd.Series(stats.sem(ewret_t['long_short']))
    mom_ser=mom_ser.T.reset_index(drop= False)
    mom_ser=mom_ser.rename(columns={'index': 'momr', 0: 'std error'})

    # T-Value and P-Value
    t_losers = pd.Series(stats.ttest_1samp(ewret_t['losers'],0.0)).to_frame().T
    t_winners = pd.Series(stats.ttest_1samp(ewret_t['winners'],0.0)).to_frame().T
    t_long_short = pd.Series(stats.ttest_1samp(ewret_t['long_short'],0.0)).to_frame().T

    t_losers['momr']='losers'
    t_winners['momr']='winners'
    t_long_short['momr']='long_short'

    t_output =pd.concat([t_winners, t_losers, t_long_short])\
        .rename(columns={0:'t-stat', 1:'p-value'})

     # Combine mean, t and p and format output
    mom_output = pd.merge(mom_mean, t_output, on=['momr'], how='inner')
    mom_output=pd.merge(mom_output, mom_ser, on='momr', how='inner')

    mom_output['mean'] = mom_output['mean'].map('{:.2%}'.format)
    mom_output['t-stat'] = mom_output['t-stat'].map('{:.2f}'.format)
    mom_output['p-value'] = mom_output['p-value'].map('{:.3f}'.format)

    return mom_output,ewret_t

In [153]:
j=1
k=6
mom,ewret_t= momentum_rets(crsp_m=crsp.copy(),jayant=jayant,j=j,k=k, gap=False,rem_excess=False,sub_risk_free=False)
mom

,momr,mean,t-stat,p-value,std error
0,winners,2.21%,3.17,0.002,0.006972
1,losers,1.89%,2.40,0.017,0.007858
2,long_short,0.33%,1.69,0.092,0.001922


In [143]:
###################
#Output Interface
##################
jk_table=False
plot=False
draw_plt =False

In [144]:
if jk_table:
    loop=12
    momentum=pd.DataFrame()
    G=[False,True]
    J=[1,3,6,9,12]
    K=[1,3,6,9,12]
    for gap in G:
        for j in J:
            for k in K:
                try:

                    mom,ewret_t= momentum_rets(crsp_m=crsp.copy(),j=j,k=k, gap=gap,rem_excess=False)

                except:
                    continue
                else:
                    mom.loc[:,"G"]=gap
                    mom.loc[:,"J"]=j
                    mom.loc[:,"K"]=k
                    momentum=pd.concat([momentum,mom],axis=0)
    momentum.to_csv('mom_op_'+str(loop) +".csv")

In [145]:
def drawdown(return_series: pd.Series,rescale_fac= False):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    if rescale_fac:
        wealth_index= -1* wealth_index
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})


In [146]:
if draw_plt:
    draw= drawdown(ewret_t['long_short'],rescale_fac=False).Drawdown
    draw.plot(figsize=(12,9),color='r',style="--")

In [147]:
if plot:
    ewret_t1['cumret_long_short'].plot(figsize=(12,6),style="--",color="red",secondary_y=True)
    ewret_t0['cumret_long_short'].plot(figsize=(12,6),style="--",color="black")
    ewret_t_['cumret_long_short'].plot(figsize=(12,6),style="--",color="blue")

# Notes and Discussions for reference of the programmer
-----------------------

## Output File 4
1. BSE price momentum based on JT 93 remove bottom 5% Market Cap
2. BSE~NSE price momentum based on JT 93 remove bottom 5% Market Cap
3. BSE price momentum based on JT 93 remove bottom 5% Market Cap most liquid decile
4. BSE price momentum based on JT 93 remove bottom 5% Market Cap least liquid decile
5. BSE~NSE price momentum based on JT 93 remove bottom 5% Market Cap least liquid decile
6. BSE~NSE price momentum based on JT 93 remove bottom 5% Market Cap most liquid decile
7. Penny stocks on bottom 25%  Market Cap based on JT 93
8. BSE~NSE price momentum based on JT 93
9. BSE price momentum based on JT 93 remove bottom 5% Market Cap most liquid decile Long term returns
10. BSE price momentum based on JT 93 remove bottom 5% Market Cap Long term returns
11. BSE price momentum based on JT 93 remove bottom 5% Market Cap least liquid decile Long term returns
12. Penny stocks on bottom 10%  Market Cap based on JT 93

## Output File 3

1. BSE price momentum without filtering
2. NSE price momentum without filtering
3. BSE~NSE price momemtum without filteringe
4. BSE price momentum JK Table
5. BSE~NSE price momentum JK Table
6. NSE price momentum JK Table
7. NSE price momentum with Market filter at 10%
8. BSE price momentum with Market filter at 10%
9. BSE~NSE price momentum with Market filter at 10%
10. BSE price momentum JK Table from 2000
11. BSE~NSE price momentum JK Table from 2000
12. NSE price momentum JK Table from 2000
13. BSE price momentum with Market filter at 10% from 2000
14. BSE~NSE price momentum with Market filter at 10% from 2000
15. NSE price momentum with Market filter at 10% from 2000
16. BSE price momentum on penny stocks below Rs. 10
17. BSE~NSE price momentum on penny stocks below Rs. 10
18. NSE price momentum on penny stocks below Rs. 10

## Output File 2
1. Unconditional without any filtering wins=0.01, remove bottom decile of Market Cap, remove excess returns decile
2. Unconditional without any filtering wins=0.01
3. Unconditional without any filtering wins=0.01, remove bottom 10% Market Cap by number
4. -> Unconditional without any filtering wins=0.01, remove bottom 25% Market Cap by number
5. Unconditional without any filtering wins=0.01, remove penny stocks under Rs 10
6. -> Most Liquid Decile wins=0.01, remove bottom decile of Market Cap
7. -> Least Liquid Decile wins=0.01, don't use cumret in the bottom decile
8. Penny Stocks in Market Cap bottom decile wins=0.01, don't use cumret in the bottom decile
9. Penny Stocks in Market Cap bottom decile wins=0.01
10. Penny Stocks in Market Cap bottom quintile 
11. Penny Stocks in Market Cap bottom quintile, don't use cumret in the bottom decile
12. Penny Stocks in Market Cap bottom quintile  wins=0.01
13. Penny Stocks in Market Cap bottom quintile wins=0.01 , don't use cumret in the bottom decile

## Output File 1

1. Unconditional without any filtering
2. Decile is liquid without any filtering
3. Decile is not liquid without any filtering -> Output Found with corrections
4. Unconditional with penny remove at 10
5. Decile is liquid with penny remove at 10
6. Decile is not liquid with penny remove at 10 -> Output by removing outliers
7. Unconditional with penny remove at 30
8. Liquid with penny remove at 30
9. Not liquid with penny remove at 30 -> Output by removing outliers
10. Unconditional with marketcap remove bottom 10
11. Liquid remove bottom 10 marketcap
12. Not liquid with marketcap remove bottom 10 marketcap -> Output Found with corrections
13. Unconditional with remove marketcap bottom 25
14. Liquid with remove marketcap bottom 25
15. Not liquid with marketcap remove bottom 25-> Output Found with corrections
16. Unconditional with marketcap remove 10 and penny 10
17. Liquid with marketcap remove 10 and penny 10
18. Not Liquid with marketcap remove 10 and penny 10 -> No Output as deciles not forming